# Frame from Video

This code is developed to receive in input youtube videos, and extract from them specific images.

To do this, pytube3 package is needed, and thus, installed.

The extractions follows the following step:

read a row from the csv file representing_


* video id, 
* image timestamp
* bounding box coordinates targeting an action inside the scene
* name of the action.

The video id is taken and used to compose the full url linking to the video. Then the video is dowloaded and saved into the notebook. 
From the downloaded video the specific image is extracted, and stored in the dataset folder, also a csv file is created representing the path of the image, the coordinates of the bounding box, and the action's name.

The final csv file is ready to be used into the Keras Retinanet implementation.

In [ ]:
# this is to install the package that allows to download videos
!pip install pytube3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pytube
import time
import pandas as pd
import requests

from IPython.display import clear_output
from pytube import YouTube
# misc
import csv
import os
import shutil
import math
import datetime
# plots
import matplotlib.pyplot as plt
%matplotlib inline
# image operation
import cv2

from google.colab.patches import cv2_imshow

# Code to prepare the CSV file 
The following code opens the csv AVA dataset file, and extract all the relevant rows to be put into another file, from where I take each video and retrieve the key image 

#The following code downloads and saves images from youtube videos

In [ ]:
# reading csv file from where to read information to retrieve the data 

filename = 'AVA_annotations_longer.csv'
count = 0 # this counts the lines read
positive_count = 0 # this counts the lines added to the annotation definitive file
track_video_name = "" # track the video name, if is the same of rìthe preceding row, is not downloaded
response = "" # track the http response from the server  

# the count in the file reading is used to start from the desired line
data = pd.read_csv(filename, header=None, sep=',', skiprows=count)
    
for row in range(len(data)):
  # for each row the information are printed and saved in the variables
  print("fields of row  ", row, " are: \n", data.loc[row], "\n")

  IMAGE_NAME = data.loc[row,0] + '.jpg'
  VIDEO_URL = 'https://www.youtube.com/watch?v='+str(data.loc[row,0])
  KEYFRAME_TIME = data.loc[row,1]
  X_min = float(data.loc[row,2])
  Y_min = float(data.loc[row,3])
  X_MAX = float(data.loc[row,4])
  Y_MAX = float(data.loc[row,5])
  ACTION_ID = int(data.loc[row,6])
  ACTION = ""

  print("x min is: ", X_min)
  print("y min is: ", Y_min)
  print("X MAX is: ", X_MAX)
  print("Y MAX is: ", Y_MAX, "\n")

  #---------------------------------------------------
  # here I check if the action is present in the classes file, and I print it
  with open('/content/drive/My Drive/Sapienza/Elective_Pirri/datasets/AVA_project/ava_actions.csv') as actions_file:
      csv_read = csv.reader(actions_file, delimiter=',')
      line_count = 0
      for row in csv_read:
        # is the row has all the informations, then I select it and save the required fields in a second csv file.
          if (int(row[1]) == ACTION_ID):
            ACTION = row[0]

  print("action is: ", ACTION, "\n")
  # the try except structure is used in order to not stop the code if some video
  # generates error being not available, or if the youtube server download limit is reached
  try:
    # if the video is already downloaded another http reques is not needed
    if (VIDEO_URL != track_video_name):
      video = YouTube(VIDEO_URL)

      video.streams.filter(file_extension = "mp4").all()

      stream = video.streams.get_highest_resolution()
      stream.download(filename='video')

      track_video_name = VIDEO_URL

    else:
      print("video was already present \n")
      

    positive_count = positive_count + 1 #video was ok so I increment the extracted lines counter

    # the timestamp is computed in milliseconds, as required from the cap.set() function
    TIME = float(KEYFRAME_TIME) * 1000
    print("time is :", TIME, "\n")

    path = '/content/drive/My Drive/dataset'
    cap = cv2.VideoCapture('video.mp4')
    cap.set(cv2.CAP_PROP_POS_MSEC,TIME)      # Go to the specified milli sec. position
    ret,frame = cap.read()                   # Retrieves the frame at the specified second
    cv2.imwrite(os.path.join(path , IMAGE_NAME), frame)
    #cv2.imwrite(IMAGE_NAME, frame)          # Saves the frame as an image
    #cv2_imshow(frame)           # Displays the frame on screen
    cv2.waitKey()                            # Waits For Input
    # dimension of the image is extracted and used to compute 
    # the right coordinates of the bounding box
    dimensions = frame.shape

    print(dimensions)

    X_min = X_min * dimensions[1]
    X_MAX = X_MAX * dimensions[1]
    Y_min = Y_min * dimensions[0]
    Y_MAX = Y_MAX * dimensions[0]

    X_min = int(round(X_min, 0))
    X_MAX = int(round(X_MAX, 0))
    Y_min = int(round(Y_min, 0))
    Y_MAX = int(round(Y_MAX, 0))

    # represents the top left corner of rectangle 
    start_point = (X_min, Y_min) 
    # represents the bottom right corner of rectangle 
    end_point = (X_MAX, Y_MAX) 

    # Blue color in BGR 
    color = (255, 0, 0) 
    # Line thickness of 2 px 
    thickness = 2

    # Draw a rectangle with blue line borders of thickness of 2 px 
    image = cv2.rectangle(frame, start_point, end_point, color, thickness) 

    cv2_imshow(image)           # Displays the frame on screen
    cv2.waitKey()                            # Waits For Input

    dimensions = frame.shape

    print(dimensions)

    # printing the coordinates of the bounding box
    print("x min is: ", X_min, " and Y min is: ", Y_min, "\n")
    print("X MAX is: ", X_MAX, "and Y MAX is: ", Y_MAX)

    with open('/content/drive/My Drive/AVA_annotations.csv', 'a+') as final_annotator:
      final_annotator = csv.writer(final_annotator, delimiter=',')
      #line = [IMAGE_NAME, X_min, Y_min, X_MAX, Y_MAX, ACTION_ID]
      final_annotator.writerow(['dataset/'+IMAGE_NAME, X_min, Y_min, X_MAX, Y_MAX, ACTION])



  except:
    response = requests.get(VIDEO_URL)
    print("response is: ", response)

    print("error on this video, skip to the next \n")

  #sleep introduced in order to avoid problems with too many requests
  time.sleep(3)

  # here I write down the last row of the CSV that has been read
  f = open("/content/drive/My Drive/track_of_work.txt", "w")
  writing = "last read line: "+ str(count)+ "\n" # Counter of last working line from the file
  f.write(writing)
  f.close()

  # ------------------------------------

  count = count+1

  # I clear the terminal after some iterations concluded
  if(count % 3 == 0):
    clear_output()

  # if the server blocked the youtube access due to "too many requests" I stop the loop
  if ("429" in str(response)):
    break
  # --------------------

  # if I end to collect data the loop stops
  if (count > 1155):
    break

print("images extraction ended!!! \n")